# Implementação ACO - Problema do Caixeiro Viajante

In [2]:
# Célula de imports

import numpy as np
import random 
import matplotlib.pyplot as plt
import pandas as pd

### Planilha com a distância de cada cidade

In [12]:
# Célula para ler o planilha com as distâncias
df = pd.read_csv('distancia_matrix.csv', header=None)

cities_distances = df

cities_distances

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,66.0,103.0,58.0,151.0,210.0,231.0,188.0,170.0,182.0,63.0,170.0,187.0,218.0,111.0,82.0,134.0,218.0,248.0,135.0
1,66.0,0.0,118.0,138.0,55.0,83.0,245.0,236.0,78.0,171.0,96.0,121.0,244.0,57.0,60.0,212.0,101.0,214.0,194.0,247.0
2,103.0,118.0,0.0,242.0,190.0,239.0,80.0,68.0,248.0,128.0,71.0,131.0,68.0,182.0,193.0,158.0,225.0,99.0,189.0,86.0
3,58.0,138.0,242.0,0.0,104.0,186.0,206.0,56.0,172.0,178.0,204.0,197.0,185.0,236.0,79.0,65.0,123.0,53.0,227.0,194.0
4,151.0,55.0,190.0,104.0,0.0,215.0,181.0,236.0,166.0,207.0,53.0,82.0,205.0,200.0,234.0,244.0,58.0,152.0,244.0,249.0
5,210.0,83.0,239.0,186.0,215.0,0.0,142.0,121.0,156.0,70.0,81.0,64.0,163.0,131.0,222.0,176.0,58.0,148.0,186.0,89.0
6,231.0,245.0,80.0,206.0,181.0,142.0,0.0,133.0,222.0,227.0,79.0,133.0,220.0,88.0,59.0,127.0,143.0,225.0,125.0,246.0
7,188.0,236.0,68.0,56.0,236.0,121.0,133.0,0.0,143.0,175.0,241.0,164.0,212.0,92.0,110.0,113.0,182.0,53.0,56.0,99.0
8,170.0,78.0,248.0,172.0,166.0,156.0,222.0,143.0,0.0,100.0,55.0,130.0,113.0,211.0,162.0,79.0,152.0,245.0,164.0,239.0
9,182.0,171.0,128.0,178.0,207.0,70.0,227.0,175.0,100.0,0.0,233.0,153.0,235.0,238.0,239.0,238.0,180.0,219.0,167.0,237.0


### PARÂMETROS GLOBAIS

In [17]:
NUM_CITIES = len(df)
EVAPORATION_RATE = 0.5
PHEROMONE_RATE = 100
ALPHA = 2
BETA = 3

### CONSTRUÇÃO DE CAMINHOS

Para a realização do algoritmo, é necessário definir, inicialmente, o caminho que cada formiga irá percorrer a partir de sua cidade inicial. O calcúlo da probabilidade da formiga visitar a cidade vizinha é dada pela seguinte fórmula:

$$
p_{ij}^k = 
\frac{
    (\tau_{ij})^\alpha \cdot (\eta_{ij})^\beta
}{
    \sum\limits_{l \in \mathcal{N}_i^k} (\tau_{il})^\alpha \cdot (\eta_{il})^\beta
}, \quad \text{se } j \in \mathcal{N}_i^k
$$

onde 

p^kij =  probabilidade da formiga k ir de i para j.

nij = 1/distância ij

tij = quantidade de feromônio na aresta (i,j)(i,j).

Nik = ​ é o conjunto de cidades ainda não visitadas pela formiga kk a partir de i.

α = controla a importância do feromônio.

β = controla a importância da heurística.

In [18]:
# Função que calcula a atratividade de cada aresta

def calculateAtrac(origin, pheMatrix, distMatrix, not_visited):

    atracDict = {}

    for destiny in not_visited:

        pheromonEdge = pow(pheMatrix[origin][destiny], ALPHA)
        distEdge =     pow((1/distMatrix[origin][destiny]), BETA) 

        atracDict[destiny] = pheromonEdge * distEdge
    
    return atracDict

In [19]:
# Algoritmo para encontrar o "melhor" caminho a partir de um ponto inicial
'''
Recebe como parâmetros:
- Cidade Inicial (int)
- Matriz com o valor de feromônios de cada aresta
- Matriz de distâncias
'''

def findPath(initialCity, pheMatrix, distMatrix):
    path = []
    not_visited = list(range(NUM_CITIES))

    # Informa que a cidade inicial ja foi visitada
    path.append(initialCity)
    not_visited.remove(initialCity)


    # Enquanto todas a cidades não forem visitadas
    while not_visited:
        probabilities_list = []

        # Calcula a atratividade de cada cidade a partir do ponto inicial
        atrac_dict = calculateAtrac(initialCity, pheMatrix, distMatrix, not_visited)

        sum_atrac = 0
        # Calcula a soma das atratividades
        sum_atrac = sum(atrac_dict.values())

        # Calcula a probabilidade para cada cidade
        for city in not_visited:
            # Lista que armazena a cidade destino + sua probabilidade
            prob_list = []

            # Calcula a probabilidade da cidade
            cityProb = atrac_dict[city]/sum_atrac
            # Salva valores
            prob_list.append(city)
            prob_list.append(cityProb)

            # Salva tupla na lista com todas as probabilidades
            probabilities_list.append(prob_list)

        # Utiliza método de sorteio proporcional para selecionar o prox caminho
        cities, probs = zip(*probabilities_list)
        next_city = random.choices(cities, weights=probs, k=1)[0]

        # Adiciona essa cidade no caminho final
        path.append(next_city)
        # Marca a cidade como visitada
        not_visited.remove(next_city)
        # Define a cidade como sendo a inicial
        initialCity = next_city

    # Retorna o caminho final
    return path   
        



In [22]:
# Célula para a realização de testes

# Cria uma matriz com valores de feromonios igual a 1
pheMatrix = np.ones((NUM_CITIES, NUM_CITIES))

# Calcula caminhos partindo de cada cidade
for i in range(NUM_CITIES):
    path = findPath(i, pheMatrix, cities_distances)
    print(f'Caminho cidade [{i}]: {path}')

Caminho cidade [0]: [0, 4, 16, 5, 12, 13, 1, 8, 10, 6, 2, 7, 17, 14, 15, 3, 11, 19, 18, 9]
Caminho cidade [1]: [1, 8, 15, 18, 17, 10, 2, 19, 14, 3, 7, 11, 5, 9, 4, 0, 16, 12, 13, 6]
Caminho cidade [2]: [2, 12, 1, 14, 7, 3, 17, 10, 0, 15, 6, 13, 16, 5, 9, 8, 18, 11, 19, 4]
Caminho cidade [3]: [3, 17, 7, 2, 12, 14, 1, 4, 16, 5, 10, 6, 18, 15, 0, 8, 9, 13, 11, 19]
Caminho cidade [4]: [4, 10, 11, 14, 18, 7, 19, 2, 1, 13, 6, 9, 3, 15, 0, 17, 16, 5, 8, 12]
Caminho cidade [5]: [5, 9, 15, 14, 16, 4, 3, 17, 10, 1, 8, 13, 6, 12, 2, 11, 0, 7, 18, 19]
Caminho cidade [6]: [6, 15, 14, 1, 13, 12, 2, 10, 11, 5, 16, 4, 3, 17, 7, 19, 0, 9, 8, 18]
Caminho cidade [7]: [7, 3, 15, 18, 10, 8, 1, 5, 9, 0, 2, 6, 13, 12, 11, 19, 14, 17, 4, 16]
Caminho cidade [8]: [8, 15, 7, 10, 17, 2, 19, 14, 6, 13, 1, 12, 5, 16, 4, 3, 0, 11, 18, 9]
Caminho cidade [9]: [9, 5, 14, 10, 15, 18, 1, 8, 16, 4, 3, 0, 2, 6, 7, 19, 11, 17, 13, 12]
Caminho cidade [10]: [10, 5, 16, 4, 11, 1, 13, 12, 2, 7, 18, 14, 0, 3, 17, 8, 15, 6, 9, 19